# The Cannon Workshop
### Import important python packages

In [2]:
import numpy as np
import pandas as pd
import thecannon as tc
from astropy.io import fits

In [5]:
labels = pd.read_csv('./labels.csv')

In [14]:
",".join(list(labels['sobject_id'].values.astype(str)))

'170506005401022,150411006101146,150428001101099,151110001601017,151008002101028,170507011101145,170510006301184,160403004201383,170510005801159,150427004801335,150427004802335,161006002601192,170507011101208,140810002201122,150607004102167,161006002601105,170506006401077,160529005901089,170510006301123,170506005901215,170603006601077,150411005601097,170506005901120,150827003401348,160531005101034,160529005901212,150413005601103,170507011101192,150427004802343,150427004801343,150427004801202,150427004802202,170830002801060,170507010601226,161013001601132,170506004901310,161013001602068,170507011101223,170510006301205,171106001901385,150604003902278,150427003302054,140707000601063,170603007201194,170507011101336,170507011101152,170830002801223,161013001602084,160529005901189,160816001701256,150411005101225,150427004302344,160531004601279,170507010601003,161006002601091,170603007201260,170507010601374,150607003601126,170509004201094,170507011101161,150603004801333,150827003401045,1705060